In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_ini=pd.read_csv('titanic.csv')

In [3]:
for feature in df_ini.columns[:]:
    if df_ini[feature].dtype!='object':        
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].median(),df_ini[feature])
    else:
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].value_counts().index[0],df_ini[feature])
df_aim=df_ini

In [4]:
def reol(feature1):
    IQR=df_aim[feature1].quantile(0.75)-df_aim[feature1].quantile(0.25)
    lbound=df_aim[feature1].quantile(0.25)-(IQR*1.5)
    ubound=df_aim[feature1].quantile(0.75)+(IQR*1.5)
    df_aim.loc[df_aim[feature1]<=lbound,feature1]=lbound
    df_aim.loc[df_aim[feature1]>=ubound,feature1]=ubound
df_aol=df_aim

In [5]:
for feature in df_aim.columns[:]:
    if df_aim[feature].dtype!='object':
        reol(feature)

In [6]:
df_aol_cont1=df_aol.select_dtypes(exclude=[object])

In [7]:
df_aol_cont2=df_aol_cont1.drop(columns='Pclass',axis=1) #Dropping categorical ordinal features

In [8]:
df_aol_cont3=df_aol_cont2.drop('Survived',axis=1)#varience threshold can be applied for independent features only

In [9]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_aol_cont3)

VarianceThreshold(threshold=0)

In [10]:
const_features=[column for column in df_aol_cont3.columns\
               if column not in df_aol_cont3.columns[var_thres.get_support()]]

In [11]:
df_aol=df_aol.drop(const_features,axis=1)#Dropping constant features

In [12]:
df_aol_cont3=df_aol_cont3.drop(columns=const_features,axis=1)

In [13]:
X=df_aol_cont3
y=df_aol_cont2['Survived']

In [14]:
def pcc(dataset,threshold):
    dropset=set()
    cor_mat=dataset.corr()
    for i in range(len(cor_mat.columns)):
        for j in range(i):
            if abs(cor_mat.iloc[i,j])>=threshold:
                cor_feat=cor_mat.columns[i]
                dropset.add(cor_feat)
    return dropset 
dropset1=pcc(X,0.85)
print("The Features to be Dropped are...")
dropset1

The Features to be Dropped are...


set()

In [15]:
df_aol=df_aol.drop(columns=dropset1,axis=1)

In [16]:
def enc(feature1):
    key_labels=df_aol[feature1].unique()
    dict={k:i for i,k in enumerate(key_labels,0)}
    df_aol[feature1]=df_aol[feature1].map(dict)

In [17]:
for feature in df_aol.columns[:]:
    if df_aol[feature].dtype=='object':
        enc(feature)
df_aec=df_aol

In [18]:
from sklearn.feature_selection import mutual_info_classif
X=df_aec.drop(columns='Survived',axis=1)
y=df_aec['Survived']
mut_inf_data=mutual_info_classif(X,y,random_state=0)
mut_inf_modif=pd.Series(mut_inf_data,index=X.columns)
mut_inf_sorted=mut_inf_modif.sort_values(ascending=False)
dict=mut_inf_sorted.to_dict()

In [19]:
droplist=[]
for key in dict:
    if dict[key]<0.001:
        droplist.append(key)

In [20]:
df_aec=df_aec.drop(columns=droplist,axis=1)

In [21]:
df_aim_cate1=df_aim.select_dtypes(include=[object])#For imbalanced data check,include all categorical features+Dep. feature

In [22]:
df_aim_cate2=df_aim[['Pclass','Survived']].copy()#include categorical ordinal feature and
df_aim_cate=pd.concat([df_aim_cate1,df_aim_cate2],axis=1)#dependent feature in the new Data frame.

In [23]:
imb_chk_list=list()

In [24]:
for feature in df_aim_cate.columns[:]:
    length=int(len(df_aim_cate[feature].unique()))      
    if length <= 5:
        imb_chk_list.append(feature)

In [25]:
drop_list=[]#Add the independent features with single category, imbalanced data in this list(Difference of values<0.05 and >0.95)

In [26]:
df_aec=df_aec.drop(drop_list,axis=1)

In [27]:
original_list=['Survived','Pclass','Sex','SibSp','Cabin','Embarked']
log_list=['Fare']
expo_list=[]
sqroot_list=['PassengerId','Name','Ticket']
boxcox_list=[]

In [28]:
for feature in log_list:
    df_aec[feature]=np.log1p(df_aec[feature])

In [29]:
for feature in sqroot_list:
    df_aec[feature]=df_aec[feature]**(1/2)

In [30]:
df_atr=df_aec

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled=scaler.fit_transform(df_atr)

In [32]:
df_ast=pd.DataFrame(scaled,columns=df_atr.columns)

In [33]:
df_fin=df_ast

In [34]:
X=df_fin.drop(columns='Survived',axis=1)
y=df_fin['Survived']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=0)

In [37]:
from sklearn.neighbors import KNeighborsClassifier
classifier=KNeighborsClassifier()
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[145  23]
 [ 28  72]]
Accuracy Score : 0.8097014925373134
              precision    recall  f1-score   support

         0.0       0.84      0.86      0.85       168
         1.0       0.76      0.72      0.74       100

    accuracy                           0.81       268
   macro avg       0.80      0.79      0.79       268
weighted avg       0.81      0.81      0.81       268



In [50]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import reciprocal,uniform
classifier=KNeighborsClassifier()

In [51]:
param_grid={'n_neighbors':[5,7,9,11,13,15],
            'weights':['uniform','distance'],
            'metric':['minkowski','euclidean','manhattan']}            

In [52]:
grid_search=GridSearchCV(estimator=classifier,param_grid=param_grid,cv=10,
                         verbose=2,n_jobs=-1)

In [53]:
classifier.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [54]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    5.6s finished


GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [5, 7, 9, 11, 13, 15],
                         'weights': ['uniform', 'distance']},
             verbose=2)

In [55]:
best_grid=grid_search.best_estimator_
best_grid

KNeighborsClassifier(metric='manhattan', n_neighbors=11)

In [56]:
prediction=best_grid.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[145  23]
 [ 27  73]]
Accuracy Score : 0.8134328358208955
              precision    recall  f1-score   support

         0.0       0.84      0.86      0.85       168
         1.0       0.76      0.73      0.74       100

    accuracy                           0.81       268
   macro avg       0.80      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268



In [62]:
# pip install optuna
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    n_neighbors=trial.suggest_categorical('n_neighbors',[5,7,9,11,13,15])
    weights=trial.suggest_categorical('weights',['uniform','distance'])
    metric=trial.suggest_categorical('metric',['minkowski','euclidean','manhattan'])
    classifier=KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,metric=metric)
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=3).mean()
    return score

In [63]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2021-08-05 22:06:23,181] A new study created in memory with name: no-name-24cfd1b5-a457-442c-804d-df3265bbe073
[I 2021-08-05 22:06:26,370] Trial 0 finished with value: 0.7656462901028118 and parameters: {'n_neighbors': 11, 'weights': 'distance', 'metric': 'minkowski'}. Best is trial 0 with value: 0.7656462901028118.
[I 2021-08-05 22:06:28,132] Trial 1 finished with value: 0.7560463891985632 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'metric': 'manhattan'}. Best is trial 0 with value: 0.7656462901028118.
[I 2021-08-05 22:06:28,192] Trial 2 finished with value: 0.7913569924439491 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:28,231] Trial 3 finished with value: 0.7881518642388207 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:28,272] Trial 4 finished with value: 0.778

[I 2021-08-05 22:06:29,603] Trial 37 finished with value: 0.7817261241174284 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:29,651] Trial 38 finished with value: 0.7785209959123002 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:29,689] Trial 39 finished with value: 0.7560463891985632 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:29,744] Trial 40 finished with value: 0.7817261241174284 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:29,797] Trial 41 finished with value: 0.7913569924439491 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial

[I 2021-08-05 22:06:31,342] Trial 75 finished with value: 0.7913569924439491 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:31,385] Trial 76 finished with value: 0.7913569924439491 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:31,429] Trial 77 finished with value: 0.7913569924439491 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:31,469] Trial 78 finished with value: 0.7785287377678682 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 2 with value: 0.7913569924439491.
[I 2021-08-05 22:06:31,513] Trial 79 finished with value: 0.7913569924439491 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial

In [64]:
best=study.best_params

In [65]:
best

{'n_neighbors': 13, 'weights': 'uniform', 'metric': 'manhattan'}

In [66]:
classifier=KNeighborsClassifier(n_neighbors=best['n_neighbors'],weights=best['weights'],metric=best['metric'])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [67]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[150  18]
 [ 28  72]]
Accuracy Score : 0.8283582089552238
              precision    recall  f1-score   support

         0.0       0.84      0.89      0.87       168
         1.0       0.80      0.72      0.76       100

    accuracy                           0.83       268
   macro avg       0.82      0.81      0.81       268
weighted avg       0.83      0.83      0.83       268



In [68]:
# pip install hyperopt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
space={'n_neighbors':hp.choice('n_neighbors',[5,7,9,11,13,15]),
       'weights':hp.choice('weights',['uniform','distance']),
       'metric':hp.choice('metric',['minkowski','euclidean','manhattan'])}

In [69]:
def objective(space):
    classifier=KNeighborsClassifier(n_neighbors=space['n_neighbors'],weights=space['weights'],metric=space['metric'])
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=5).mean()
    return{'loss':-score,'status':STATUS_OK}

In [70]:
trials=Trials()
best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=80,trials=trials)
best

100%|███████████████████████████████████████████████| 80/80 [00:07<00:00, 10.74trial/s, best loss: -0.8041548387096775]


{'metric': 2, 'n_neighbors': 3, 'weights': 0}

In [72]:
neigh={0:5,1:7,2:9,3:11,4:13,5:15}
wei={0:'uniform',1:'distance'}
met={0:'minkowski',1:'euclidean',2:'manhattan'}

In [73]:
classifier=KNeighborsClassifier(n_neighbors=neigh[best['n_neighbors']],weights=wei[best['weights']],metric=met[best['metric']])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [74]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[145  23]
 [ 27  73]]
Accuracy Score : 0.8134328358208955
              precision    recall  f1-score   support

         0.0       0.84      0.86      0.85       168
         1.0       0.76      0.73      0.74       100

    accuracy                           0.81       268
   macro avg       0.80      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268

